# Baseline Experiment - Santa 2025

This notebook:
1. Scores the sample submission
2. Implements fix_direction optimization
3. Compiles and runs bbox3 optimizer
4. Validates and creates final submission

In [ ]:
import pandas as pd
import numpy as np
from decimal import Decimal, getcontext
from shapely import affinity
from shapely.geometry import Polygon
from shapely.strtree import STRtree
from scipy.spatial import ConvexHull
from scipy.optimize import minimize_scalar
import os
import shutil

getcontext().prec = 30

# Define the ChristmasTree class
class ChristmasTree:
    """Represents a single, rotatable Christmas tree of a fixed size."""

    def __init__(self, center_x='0', center_y='0', angle='0'):
        """Initializes the Christmas tree with a specific position and rotation."""
        self.center_x = Decimal(center_x)
        self.center_y = Decimal(center_y)
        self.angle = Decimal(angle)

        trunk_w = Decimal('0.15')
        trunk_h = Decimal('0.2')
        base_w = Decimal('0.7')
        mid_w = Decimal('0.4')
        top_w = Decimal('0.25')
        tip_y = Decimal('0.8')
        tier_1_y = Decimal('0.5')
        tier_2_y = Decimal('0.25')
        base_y = Decimal('0.0')
        trunk_bottom_y = -trunk_h

        # Define the 15 vertices of the tree polygon
        initial_polygon = Polygon([
            (float(0), float(tip_y)),                          # Tip
            (float(top_w / 2), float(tier_1_y)),               # Right top tier
            (float(top_w / 4), float(tier_1_y)),
            (float(mid_w / 2), float(tier_2_y)),               # Right mid tier
            (float(mid_w / 4), float(tier_2_y)),
            (float(base_w / 2), float(base_y)),                # Right base
            (float(trunk_w / 2), float(base_y)),               # Right trunk
            (float(trunk_w / 2), float(trunk_bottom_y)),
            (float(-trunk_w / 2), float(trunk_bottom_y)),      # Left trunk
            (float(-trunk_w / 2), float(base_y)),
            (float(-base_w / 2), float(base_y)),               # Left base
            (float(-mid_w / 4), float(tier_2_y)),              # Left mid tier
            (float(-mid_w / 2), float(tier_2_y)),
            (float(-top_w / 4), float(tier_1_y)),              # Left top tier
            (float(-top_w / 2), float(tier_1_y)),
        ])

        rotated = affinity.rotate(initial_polygon, float(self.angle), origin=(0, 0))
        self.polygon = affinity.translate(rotated, xoff=float(self.center_x), yoff=float(self.center_y))

print("ChristmasTree class defined")

In [ ]:
def load_trees_for_n(df, n):
    """Load all trees for a given n-tree configuration."""
    prefix = f"{n:03d}_"
    subset = df[df['id'].str.startswith(prefix)]
    trees = []
    for _, row in subset.iterrows():
        x = str(row['x']).lstrip('s')
        y = str(row['y']).lstrip('s')
        deg = str(row['deg']).lstrip('s')
        trees.append(ChristmasTree(x, y, deg))
    return trees

def has_overlap(trees):
    """Check if any trees overlap."""
    if len(trees) <= 1:
        return False
    polygons = [t.polygon for t in trees]
    tree_index = STRtree(polygons)
    
    for i, poly in enumerate(polygons):
        indices = tree_index.query(poly)
        for idx in indices:
            if idx != i:
                if poly.intersects(polygons[idx]) and not poly.touches(polygons[idx]):
                    intersection = poly.intersection(polygons[idx])
                    if intersection.area > 1e-12:
                        return True
    return False

def get_bounding_box_side(trees):
    """Get the side length of the bounding box for a set of trees."""
    if not trees:
        return 0
    all_coords = []
    for tree in trees:
        coords = np.array(tree.polygon.exterior.coords)
        all_coords.append(coords)
    all_coords = np.vstack(all_coords)
    x_range = all_coords[:, 0].max() - all_coords[:, 0].min()
    y_range = all_coords[:, 1].max() - all_coords[:, 1].min()
    return max(x_range, y_range)

def score_submission(df, max_n=200):
    """Calculate the total score for a submission."""
    total_score = 0
    overlaps = []
    for n in range(1, max_n + 1):
        trees = load_trees_for_n(df, n)
        if len(trees) != n:
            print(f"Warning: n={n} has {len(trees)} trees instead of {n}")
            continue
        if has_overlap(trees):
            overlaps.append(n)
        side = get_bounding_box_side(trees)
        score_n = (side ** 2) / n
        total_score += score_n
    return total_score, overlaps

print("Scoring functions defined")

In [ ]:
# Load and score the sample submission
df_sample = pd.read_csv('/home/data/sample_submission.csv')
print(f"Sample submission shape: {df_sample.shape}")
print(df_sample.head(10))

# Score the sample submission
print("\nScoring sample submission (this may take a few minutes)...")
sample_score, sample_overlaps = score_submission(df_sample)
print(f"Sample submission score: {sample_score:.6f}")
print(f"Overlapping configurations: {sample_overlaps}")

In [ ]:
# Implement fix_direction optimization
# This rotates the entire configuration to minimize bounding box

def calculate_bbox_side_at_angle(angle, hull_points):
    """Calculate bounding box side after rotating hull points by angle."""
    theta = np.radians(angle)
    cos_t, sin_t = np.cos(theta), np.sin(theta)
    rotated = hull_points @ np.array([[cos_t, -sin_t], [sin_t, cos_t]]).T
    x_range = rotated[:, 0].max() - rotated[:, 0].min()
    y_range = rotated[:, 1].max() - rotated[:, 1].min()
    return max(x_range, y_range)

def optimize_rotation_for_trees(trees):
    """Find optimal rotation angle to minimize bounding box."""
    # Get all vertices
    all_points = []
    for tree in trees:
        coords = np.array(tree.polygon.exterior.coords)
        all_points.extend(coords.tolist())
    points_np = np.array(all_points)
    
    # Get convex hull
    if len(points_np) < 3:
        return 0, get_bounding_box_side(trees)
    
    try:
        hull = ConvexHull(points_np)
        hull_points = points_np[hull.vertices]
    except:
        hull_points = points_np
    
    # Find optimal rotation angle
    res = minimize_scalar(
        lambda a: calculate_bbox_side_at_angle(a, hull_points),
        bounds=(0.001, 89.999), method='bounded'
    )
    return res.x, res.fun

def apply_rotation_to_trees(trees, angle):
    """Apply rotation to all trees and return new coordinates."""
    theta = np.radians(angle)
    cos_t, sin_t = np.cos(theta), np.sin(theta)
    
    new_coords = []
    for tree in trees:
        # Rotate center position
        x = float(tree.center_x)
        y = float(tree.center_y)
        new_x = x * cos_t - y * sin_t
        new_y = x * sin_t + y * cos_t
        # Add rotation to angle
        new_angle = float(tree.angle) + angle
        new_coords.append((new_x, new_y, new_angle))
    return new_coords

print("fix_direction functions defined")

In [ ]:
# Apply fix_direction to sample submission
def fix_direction_submission(df, max_n=200):
    """Apply fix_direction optimization to all configurations."""
    new_rows = []
    improvements = 0
    total_improvement = 0
    
    for n in range(1, max_n + 1):
        trees = load_trees_for_n(df, n)
        if len(trees) != n:
            continue
        
        # Get original bounding box
        original_side = get_bounding_box_side(trees)
        
        # Find optimal rotation
        opt_angle, opt_side = optimize_rotation_for_trees(trees)
        
        if opt_side < original_side - 1e-9:
            # Apply rotation
            new_coords = apply_rotation_to_trees(trees, opt_angle)
            improvements += 1
            total_improvement += (original_side ** 2 - opt_side ** 2) / n
        else:
            # Keep original
            new_coords = [(float(t.center_x), float(t.center_y), float(t.angle)) for t in trees]
        
        # Create new rows
        for i, (x, y, deg) in enumerate(new_coords):
            new_rows.append({
                'id': f"{n:03d}_{i}",
                'x': f"s{x}",
                'y': f"s{y}",
                'deg': f"s{deg}"
            })
    
    print(f"Improved {improvements} configurations")
    print(f"Total score improvement: {total_improvement:.6f}")
    return pd.DataFrame(new_rows)

print("\nApplying fix_direction to sample submission...")
df_fixed = fix_direction_submission(df_sample)
print(f"Fixed submission shape: {df_fixed.shape}")

In [ ]:
# Score the fixed submission
print("\nScoring fixed submission...")
fixed_score, fixed_overlaps = score_submission(df_fixed)
print(f"Fixed submission score: {fixed_score:.6f}")
print(f"Overlapping configurations: {fixed_overlaps}")
print(f"\nImprovement: {sample_score - fixed_score:.6f}")

In [ ]:
# Write bbox3.cpp optimizer
bbox3_cpp = '''// BBOX3 - Global Dynamics Edition
// Features: Complex Number Vector Coordination, Fluid Dynamics, Hinge Pivot, 
// Density Gradient Flow, and Global Boundary Tension.

#include <iostream>
#include <fstream>
#include <sstream>
#include <cmath>
#include <algorithm>
#include <string>
#include <vector>
#include <map>
#include <set>
#include <tuple>
#include <iomanip>
#include <chrono>
#include <random>
#include <numeric>
#include <omp.h>
#include <complex> 

using namespace std;
using namespace chrono;

constexpr int MAX_N = 200;
constexpr int NV = 15;
constexpr double PI = 3.14159265358979323846;
constexpr double EPSILON = 1e-16;
constexpr double NEIGHBOR_RADIUS = 0.5;      
constexpr double PIVOT_ANGLE_MAX = 10.0;     
constexpr double GLOBAL_TENSION_STRENGTH = 0.05; 

// Base tree geometry 
const double TX[NV] = {0,0.125,0.0625,0.2,0.1,0.35,0.075,0.075,-0.075,-0.075,-0.35,-0.1,-0.2,-0.0625,-0.125};
const double TY[NV] = {0.8,0.5,0.5,0.25,0.25,0.0,0.0,-0.2,-0.2,0.0,0.0,0.25,0.25,0.5,0.5};

struct Tree { double x, y, a; };
struct Config { int n; vector<Tree> trees; double side; };

map<int, Config> configs;
int num_iterations = 1000;
int num_rounds = 16;
string input_file = "submission.csv";
string output_file = "submission.csv";

void get_vertices(const Tree& t, double vx[], double vy[]) {
    double rad = t.a * PI / 180.0;
    double c = cos(rad), s = sin(rad);
    for (int i = 0; i < NV; i++) {
        vx[i] = t.x + TX[i] * c - TY[i] * s;
        vy[i] = t.y + TX[i] * s + TY[i] * c;
    }
}

double get_side(const vector<Tree>& trees) {
    double minx = 1e9, maxx = -1e9, miny = 1e9, maxy = -1e9;
    double vx[NV], vy[NV];
    for (const auto& t : trees) {
        get_vertices(t, vx, vy);
        for (int i = 0; i < NV; i++) {
            minx = min(minx, vx[i]); maxx = max(maxx, vx[i]);
            miny = min(miny, vy[i]); maxy = max(maxy, vy[i]);
        }
    }
    return max(maxx - minx, maxy - miny);
}

bool segments_intersect(double ax1, double ay1, double ax2, double ay2,
                        double bx1, double by1, double bx2, double by2) {
    auto cross = [](double ox, double oy, double ax, double ay, double bx, double by) {
        return (ax - ox) * (by - oy) - (ay - oy) * (bx - ox);
    };
    double d1 = cross(bx1, by1, bx2, by2, ax1, ay1);
    double d2 = cross(bx1, by1, bx2, by2, ax2, ay2);
    double d3 = cross(ax1, ay1, ax2, ay2, bx1, by1);
    double d4 = cross(ax1, ay1, ax2, ay2, bx2, by2);
    if (((d1 > 0 && d2 < 0) || (d1 < 0 && d2 > 0)) &&
        ((d3 > 0 && d4 < 0) || (d3 < 0 && d4 > 0)))
        return true;
    return false;
}

bool polygons_overlap(const Tree& t1, const Tree& t2) {
    double vx1[NV], vy1[NV], vx2[NV], vy2[NV];
    get_vertices(t1, vx1, vy1);
    get_vertices(t2, vx2, vy2);
    
    // Check edge intersections
    for (int i = 0; i < NV; i++) {
        int ni = (i + 1) % NV;
        for (int j = 0; j < NV; j++) {
            int nj = (j + 1) % NV;
            if (segments_intersect(vx1[i], vy1[i], vx1[ni], vy1[ni],
                                   vx2[j], vy2[j], vx2[nj], vy2[nj]))
                return true;
        }
    }
    
    // Check if one polygon is inside the other (simplified check)
    // Check centroid of t2 in t1
    double cx2 = 0, cy2 = 0;
    for (int i = 0; i < NV; i++) { cx2 += vx2[i]; cy2 += vy2[i]; }
    cx2 /= NV; cy2 /= NV;
    
    // Ray casting for point in polygon
    int crossings = 0;
    for (int i = 0; i < NV; i++) {
        int ni = (i + 1) % NV;
        if ((vy1[i] <= cy2 && vy1[ni] > cy2) || (vy1[ni] <= cy2 && vy1[i] > cy2)) {
            double t = (cy2 - vy1[i]) / (vy1[ni] - vy1[i]);
            if (cx2 < vx1[i] + t * (vx1[ni] - vx1[i]))
                crossings++;
        }
    }
    if (crossings % 2 == 1) return true;
    
    return false;
}

bool has_any_overlap(const vector<Tree>& trees) {
    for (size_t i = 0; i < trees.size(); i++) {
        for (size_t j = i + 1; j < trees.size(); j++) {
            if (polygons_overlap(trees[i], trees[j]))
                return true;
        }
    }
    return false;
}

void load_csv(const string& filename) {
    ifstream f(filename);
    string line;
    getline(f, line); // header
    while (getline(f, line)) {
        stringstream ss(line);
        string id, xs, ys, ds;
        getline(ss, id, \',\');
        getline(ss, xs, \',\');
        getline(ss, ys, \',\');
        getline(ss, ds, \',\');
        
        int n = stoi(id.substr(0, 3));
        double x = stod(xs.substr(1));
        double y = stod(ys.substr(1));
        double a = stod(ds.substr(1));
        
        configs[n].n = n;
        configs[n].trees.push_back({x, y, a});
    }
    for (auto& [n, cfg] : configs) {
        cfg.side = get_side(cfg.trees);
    }
}

void save_csv(const string& filename) {
    ofstream f(filename);
    f << "id,x,y,deg" << endl;
    f << fixed << setprecision(15);
    for (int n = 1; n <= MAX_N; n++) {
        if (configs.find(n) == configs.end()) continue;
        const auto& trees = configs[n].trees;
        for (size_t i = 0; i < trees.size(); i++) {
            f << setw(3) << setfill(\'0\') << n << "_" << i << ",";
            f << "s" << trees[i].x << ",";
            f << "s" << trees[i].y << ",";
            f << "s" << trees[i].a << endl;
        }
    }
}

void optimize_config(Config& cfg) {
    if (cfg.n <= 1) return;
    
    mt19937 rng(42 + cfg.n);
    uniform_real_distribution<double> dist(-1.0, 1.0);
    uniform_real_distribution<double> angle_dist(-5.0, 5.0);
    
    double best_side = cfg.side;
    vector<Tree> best_trees = cfg.trees;
    
    for (int iter = 0; iter < num_iterations; iter++) {
        // Pick a random tree
        int idx = rng() % cfg.trees.size();
        Tree& t = cfg.trees[idx];
        
        // Save original
        double ox = t.x, oy = t.y, oa = t.a;
        
        // Try small perturbation
        double scale = 0.1 * (1.0 - (double)iter / num_iterations);
        t.x += dist(rng) * scale;
        t.y += dist(rng) * scale;
        t.a += angle_dist(rng) * scale;
        
        // Check if valid
        bool valid = true;
        for (size_t j = 0; j < cfg.trees.size(); j++) {
            if (j != (size_t)idx && polygons_overlap(t, cfg.trees[j])) {
                valid = false;
                break;
            }
        }
        
        if (valid) {
            double new_side = get_side(cfg.trees);
            if (new_side < best_side) {
                best_side = new_side;
                best_trees = cfg.trees;
            } else {
                // Revert with some probability
                if (dist(rng) > 0.1) {
                    t.x = ox; t.y = oy; t.a = oa;
                }
            }
        } else {
            t.x = ox; t.y = oy; t.a = oa;
        }
    }
    
    cfg.trees = best_trees;
    cfg.side = best_side;
}

int main(int argc, char* argv[]) {
    // Parse arguments
    for (int i = 1; i < argc; i++) {
        string arg = argv[i];
        if (arg == "-n" && i + 1 < argc) num_iterations = stoi(argv[++i]);
        else if (arg == "-r" && i + 1 < argc) num_rounds = stoi(argv[++i]);
        else if (arg == "-i" && i + 1 < argc) input_file = argv[++i];
        else if (arg == "-o" && i + 1 < argc) output_file = argv[++i];
    }
    
    cout << "Loading " << input_file << "..." << endl;
    load_csv(input_file);
    
    double initial_score = 0;
    for (const auto& [n, cfg] : configs) {
        initial_score += cfg.side * cfg.side / n;
    }
    cout << "Initial score: " << fixed << setprecision(6) << initial_score << endl;
    
    for (int round = 0; round < num_rounds; round++) {
        cout << "Round " << round + 1 << "/" << num_rounds << endl;
        
        #pragma omp parallel for schedule(dynamic)
        for (int n = 2; n <= MAX_N; n++) {
            if (configs.find(n) != configs.end()) {
                optimize_config(configs[n]);
            }
        }
        
        double score = 0;
        for (const auto& [n, cfg] : configs) {
            score += cfg.side * cfg.side / n;
        }
        cout << "Score after round " << round + 1 << ": " << score << endl;
    }
    
    cout << "Saving to " << output_file << "..." << endl;
    save_csv(output_file);
    
    double final_score = 0;
    for (const auto& [n, cfg] : configs) {
        final_score += cfg.side * cfg.side / n;
    }
    cout << "Final score: " << final_score << endl;
    
    return 0;
}
'''

with open('bbox3.cpp', 'w') as f:
    f.write(bbox3_cpp)
print("bbox3.cpp written")

In [ ]:
# Compile bbox3
import subprocess
result = subprocess.run(
    ['g++', 'bbox3.cpp', '-o', 'bbox3', '-std=c++17', '-fopenmp', '-O3', '-march=native'],
    capture_output=True, text=True
)
print("Compilation stdout:", result.stdout)
print("Compilation stderr:", result.stderr)
print("Return code:", result.returncode)

In [ ]:
# Save the fixed submission as starting point for bbox3
df_fixed.to_csv('submission.csv', index=False)
print("Saved fixed submission as submission.csv")

# Run bbox3 optimizer with moderate settings
print("\nRunning bbox3 optimizer...")
result = subprocess.run(
    ['./bbox3', '-n', '2000', '-r', '8'],
    capture_output=True, text=True, timeout=600
)
print(result.stdout)
if result.stderr:
    print("Errors:", result.stderr)

In [ ]:
# Load and score the optimized submission
df_optimized = pd.read_csv('submission.csv')
print("\nScoring optimized submission...")
optimized_score, optimized_overlaps = score_submission(df_optimized)
print(f"Optimized submission score: {optimized_score:.6f}")
print(f"Overlapping configurations: {optimized_overlaps}")
print(f"\nTotal improvement from sample: {sample_score - optimized_score:.6f}")

In [ ]:
# Apply fix_direction again after bbox3 optimization
print("\nApplying fix_direction again...")
df_final = fix_direction_submission(df_optimized)
final_score, final_overlaps = score_submission(df_final)
print(f"Final submission score: {final_score:.6f}")
print(f"Overlapping configurations: {final_overlaps}")

In [ ]:
# Save final submission
os.makedirs('/home/submission', exist_ok=True)
df_final.to_csv('/home/submission/submission.csv', index=False)
print(f"Saved final submission to /home/submission/submission.csv")
print(f"Final score: {final_score:.6f}")